Chrome is a memory hog, this notebook works best in other browsers.

In [3]:
import pickle
import sys
sys.path.append('python')
import clusterOutliers
with open('data/output/Q8_sample.coo','rb') as file:
    Q8_coo = pickle.load(file)

In [4]:
import numpy as np
from astropy.io import fits
import pandas as pd

data = Q8_coo.data
Quarter = 8
pca = Q8_coo.reductions['PCA90']
x = pca.iloc[:,0]
y = pca.iloc[:,1]
files = data.index
labels = np.array(['KIC '+str(int(lc[4:13])) for lc in files])

In [3]:
Q8_coo.scores

,k1_1000x10,DB_outliers
kplr011241837-2011073133259_llc.fits,0.003211,0
kplr011821279-2011073133259_llc.fits,0.003997,0
kplr006342144-2011073133259_llc.fits,0.002002,0
kplr010355036-2011073133259_llc.fits,0.003108,0
kplr007757721-2011073133259_llc.fits,0.001226,0
...,...,...
kplr011721462-2011073133259_llc.fits,0.001952,0
kplr007499535-2011073133259_llc.fits,0.001459,0
kplr008960179-2011073133259_llc.fits,0.006952,0
kplr010861822-2011073133259_llc.fits,0.001356,0


In [5]:
scores = Q8_coo.scores['k1_1000x10']
dblabels = Q8_coo.scores['DB_outliers']
dblabels_as_string = np.array(dblabels,dtype=str)

In [1]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [7]:
import pickle
import sys
sys.path.append('python')
import clusterOutliers
import numpy as np
from quarterTools import read_kepler_curve, pca_red
import pandas as pd
from lightkurve import search_lightcurvefile

from bokeh.io import curdoc
from bokeh.plotting import figure
from bokeh.layouts import row,gridplot,column
from bokeh.models import ColumnDataSource, Circle
from bokeh.models.widgets import Button, Select
from bokeh.events import Tap
from bokeh.palettes import Colorblind4,Category10
from bokeh.transform import factor_cmap, linear_cmap, log_cmap

def modify_doc(doc):
    
    ##### Data Import #####
    with open('data/output/Q8_sample.coo','rb') as file:
        Q8_coo = pickle.load(file)

    data               = Q8_coo.data
    Quarter            = 8
    pca                = Q8_coo.reductions['PCA90']
    x                  = pca.iloc[:,0]
    y                  = pca.iloc[:,1]
    files              = data.index
    labels             = np.array(['KIC '+str(int(lc[4:13])) for lc in files])
    dblabels           = Q8_coo.scores['DB_outliers']
    dblabels_as_string = np.array(dblabels,dtype=str)
    scores             = Q8_coo.scores['k1_1000x10']

    ##### Bokeh Setup #####
    # Tools to use
    TOOLS     = "pan,wheel_zoom,reset,tap,box_select,poly_select"
    palette   = Colorblind4[0], Colorblind4[1], Colorblind4[3] # Color palette for outlier cmap
    alphas    = {1:1,2:.9,3:.8,4:.75,5:.7,6:.65,7:.6,8:.55,9:.5,10:.5}
    cmap_dict = {
        "Base"            : 'red',
        "Outliers"        : factor_cmap('labels',palette = palette,factors = np.unique(dblabels_as_string)),
        "Scores - Linear" : linear_cmap('scores','Viridis256',0,1),
        "Scores - Log"    : log_cmap('scores','Viridis256',1E-4,1E0)
    }

    def update_colors(attrname, old, new):
        cmap = cmap_dict[select.value]
        scatter.fill_color = cmap
        sg.fill_color = nsg.fill_color = cmap
        #scatter.selection_glyph.fill_color=cmap
        #scatter.nonselection_glyph.fill_color=cmap

    # Bokeh data
    # create a column data source for the plots to share
    s1 = ColumnDataSource(data={
        'x'     : x,
        'y'     : y,
        'desc'  : labels,
        'files' : files,
        'labels': dblabels_as_string,
        'scores': scores
    }) 

    # Set up widgets
    button = Button(label='Plot Selected')

    select = Select(title="Color Scheme:", value="Base", options=["Base", "Outliers", "Scores - Linear", "Scores - Log"])
    select.on_change('value',update_colors)

    # create a new plot and add a renderer
    left = figure(tools=TOOLS, plot_width=1000, plot_height=600, title=None)
    cmap = palette[0]
    scatter = Circle(x='x', y='y', fill_color=cmap, line_color=None, size=4)
    sg = Circle(
        fill_color= cmap,
        line_color=None,  
        size=4,
    )
    nsg = Circle(
        line_color=None, 
        fill_color=cmap, 
        size=4, 
        fill_alpha=.4,
    )
    left.add_glyph(s1,scatter,selection_glyph=sg,nonselection_glyph=nsg)

    # Set up layouts and add to document
    inputs = row(button,select)
    #p = gridplot([[left,right]]) #future planning
    layout = column(inputs, left)
    doc.add_root(layout)
    
show(modify_doc,notebook_url="http://localhost:8888")

In [18]:
# sometimes it's hard to tell if bokeh is updating or stuck, running a dummy cell will prompt errors sometimes
True

True

In [105]:
p = figure()
source = ColumnDataSource(data={
    'x'      : [[0,1],[0,1],[0,1,2]],
    'y'      : [[0,1],[1,0],[.5,.5,.5]],
    'colors' : Category10[10][:3],
})
lp = p.multi_line('x','y',color='colors',source=source)
show(p)

In [58]:
import pickle
import sys
sys.path.append('python')
import clusterOutliers
import numpy as np
from quarterTools import colors_for_plot
import pandas as pd
from lightkurve import search_lightcurvefile

from bokeh.io import curdoc
from bokeh.plotting import figure
from bokeh.layouts import row,gridplot,column
from bokeh.models import ColumnDataSource, Circle
from bokeh.models.widgets import Button, Select
from bokeh.events import Tap
from bokeh.palettes import Colorblind4,Category10
from bokeh.transform import factor_cmap, linear_cmap, log_cmap

def modify_doc(doc):
    
    ##### Data Import #####
    with open('data/output/Q8_sample.coo','rb') as file:
        Q8_coo = pickle.load(file)

    data               = Q8_coo.data
    Quarter            = 8
    pca                = Q8_coo.reductions['PCA90']
    x                  = pca.iloc[:,0]
    y                  = pca.iloc[:,1]
    files              = data.index
    labels             = np.array(['KIC '+str(int(lc[4:13])) for lc in files])
    dblabels           = Q8_coo.scores['DB_outliers']
    dbcolors           = colors_for_plot(dblabels,'color_blind')
    dblabels_as_string = np.array(dblabels,dtype=str)
    scores             = Q8_coo.scores['k1_1000x10']
    lin_colors         = colors_for_plot(-scores,'viridis')
    log_colors         = colors_for_plot(-np.log(scores),'viridis')
    

    ##### Bokeh Setup #####
    # Tools to use
    TOOLS     = "pan,wheel_zoom,reset,tap,box_select,poly_select"
    palette   = Colorblind4[0], Colorblind4[1], Colorblind4[3] # Color palette for outlier cmap
    alphas    = {1:1,2:.9,3:.8,4:.75,5:.7,6:.65,7:.6,8:.55,9:.5,10:.5}
    cmap_dict = {
        "Base"            : palette[0],
        "Outliers"        : [to_hex(c) for c in dbcolors],
        "Scores - Linear" : [to_hex(c) for c in lin_colors],
        "Scores - Log"    : [to_hex(c) for c in log_colors],
    }

    def update_colors(attrname, old, new):
        if select.value == "Base":
            s1.data['colors'] = [cmap_dict[select.value]]*len(s1.data['x'])
        else:
            s1.data['colors'] = cmap_dict[select.value]
        #scatter.fill_color = cmap
        #sg.fill_color = nsg.fill_color = cmap
        #scatter.selection_glyph.fill_color=cmap
        #scatter.nonselection_glyph.fill_color=cmap

    # Bokeh data
    # create a column data source for the plots to share
    s1 = ColumnDataSource(data={
        'x'     : x,
        'y'     : y,
        'desc'  : labels,
        'files' : files,
        'labels': dblabels_as_string,
        'scores': scores,
        'colors': [cmap_dict['Base']]*len(x)
    }) 

    # Set up widgets
    button = Button(label='Plot Selected')

    select = Select(title="Color Scheme:", value="Base", options=["Base", "Outliers", "Scores - Linear", "Scores - Log"])
    select.on_change('value',update_colors)

    # create a new plot and add a renderer
    left = figure(tools=TOOLS, plot_width=1000, plot_height=600, title=None)
    scatter = left.circle(x='x', y='y', fill_color='colors', line_color=None, size=4, source=s1)

    # Set up layouts and add to document
    inputs = row(button,select)
    #p = gridplot([[left,right]]) #future planning
    layout = column(inputs, left)
    doc.add_root(layout)
    
show(modify_doc,notebook_url="http://localhost:8888")

In [14]:
np.log(scores)

kplr011241837-2011073133259_llc.fits   -5.741142
kplr011821279-2011073133259_llc.fits   -5.522272
kplr006342144-2011073133259_llc.fits   -6.213453
kplr010355036-2011073133259_llc.fits   -5.773762
kplr007757721-2011073133259_llc.fits   -6.703617
                                          ...   
kplr011721462-2011073133259_llc.fits   -6.238801
kplr007499535-2011073133259_llc.fits   -6.529726
kplr008960179-2011073133259_llc.fits   -4.968737
kplr010861822-2011073133259_llc.fits   -6.603322
kplr008462852-2011073133259_llc.fits   -3.466326
Name: k1_1000x10, Length: 20000, dtype: float64

In [43]:
lin_colors         = colors_for_plot(scores)
log_colors         = colors_for_plot(np.log(scores))

In [ ]:
lin_colors

In [52]:
p = figure()
p.circle([0,1],[0,1],size=10,fill_color=['#00bc00','#cccccc'])
show(p)

array([[  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    ,   0.    ,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    ,   0.    ,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.325

In [ ]:
##### Data Import #####
with open('data/output/Q8_sample.coo','rb') as file:
    Q8_coo = pickle.load(file)

data               = Q8_coo.data
Quarter            = 8
pca                = Q8_coo.reductions['PCA90']
x                  = pca.iloc[:,0]
y                  = pca.iloc[:,1]
files              = data.index
labels             = np.array(['KIC '+str(int(lc[4:13])) for lc in files])
dblabels           = Q8_coo.scores['DB_outliers']
dblabels_as_string = np.array(dblabels,dtype=str)
scores             = Q8_coo.scores['k1_1000x10']